In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import easyocr

model = YOLO("yolov8n.pt")
reader = easyocr.Reader(['en'])

cap = cv2.VideoCapture("match_foul_clip_2.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("goalkeeper_red_card.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
if not out.isOpened():
    print("Error: Could not open output video file!")

prev_frame = None
frame_count = 0
foul_frame_detected = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_frame is not None:
        frame_diff = cv2.absdiff(prev_frame, gray)
        motion_score = np.sum(frame_diff)
        if motion_score > 1_000_000:
            foul_frame_detected.append(frame_count)

    prev_frame = gray

    results = model(frame)
    annotated = results[0].plot()

    if frame_count in foul_frame_detected:
        print(f"Goalkeeper Foul Detected at frame {frame_count}")
        for box in results[0].boxes:
            cls_id = int(box.cls[0])
            if cls_id == 0:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                player_crop = frame[y1:y2, x1:x2]
                if player_crop.size > 0:
                    gray_crop = cv2.cvtColor(player_crop, cv2.COLOR_BGR2GRAY)
                    result = reader.readtext(gray_crop)
                    for (_, text, _) in result:
                        if text.isdigit():
                            player_number = text
                            print(f"  -> Goalkeeper Number: {player_number}")
                            cv2.putText(annotated, f"Goalkeeper Number: {player_number}", (x1, y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                            break

        cv2.putText(annotated, "Goalkeeper Foul Detected", (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.putText(annotated, "Red Card", (50, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    out.write(annotated)

cap.release()
out.release()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 256x640 18 persons, 362.0ms
Speed: 32.9ms preprocess, 362.0ms inference, 30.8ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 20 persons, 263.8ms
Speed: 4.6ms preprocess, 263.8ms inference, 2.7ms postprocess per image at shape (1, 3, 256, 640)
Goalkeeper Foul Detected at frame 2

0: 256x640 14 persons, 129.3ms
Speed: 4.1ms preprocess, 129.3ms inference, 1.7ms postprocess per image at shape (1, 3, 256, 640)
Goalkeeper Foul Detected at frame 3

0: 256x640 15 persons, 127.9ms
Speed: 4.9ms preprocess, 127.9ms inference, 2.5ms postprocess per image at shape (1, 3, 256, 640)
Goalkeeper Foul Detected at frame 4

0: 256x640 17 persons, 117.3ms
Speed: 5.5ms preprocess, 117.3ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)
Goalkeeper Foul Detected at frame 5

0: 256x640 16 persons, 162.7ms
Speed: 6.0ms preprocess, 162.7ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)
Goalkeeper Foul Detected at frame 6

0: 256x640 14 persons, 180.6ms
Speed: